## Data 전처리

### 기업 통일

2016년을 기준으로 2017,2018,2019의 기업을 통일 시킬 것이다.

2016년 이후 신규 창립기업은 모델 학습, 테스트에 사용하지 않을 것이며, 파산기업은 파산한 시점부터 모든 값들을 0으로 처리한다.

In [1]:
import pandas as pd
import numpy as np

In [2]:
standard_data = pd.read_excel('C:/KYU/2017.01.xlsx', encoding= "utf-8")

In [3]:
standard_data.head()

,Code,회사명,업종\n(대),업종\n(소),코스피코스닥,주가\n(원),시가총액\n(억),1일\n등락률\n(%),5일\n등락률\n(%),20일\n등락률\n(%),...,16년 3Q.2,16년 4Q(E).2,17년 1Q(E).2,16년4Q \n매출액,16년4Q \n영업이익,16년4Q \n순이익,16년4Q \n매출액.1,16년4Q \n영업이익.1,16년4Q \n순이익.1,16년4Q \n순이익.2
0,A005930,삼성전자,반도체 관련장비 및 부품,종합 반도체,코스피,1833000,2578652,-2.135611,-1.504567,2.230898,...,44087.90,64474.315237,66685.180236,530000.0,92000.0,NaN,521120.451352,83715.404235,64474.315237,64474.315237
1,A000660,SK하이닉스,반도체 관련장비 및 부품,종합 반도체,코스피,49300,358905,-1.988072,-0.504541,6.250000,...,5967.74,10690.789385,12736.206023,NaN,NaN,NaN,51332.259150,14568.549707,10690.789385,10690.789385
2,A005380,현대차,자동차 및 관련부품,완성차,코스피,150500,331516,1.006711,0.000000,7.885305,...,10614.40,13513.247779,14784.204410,NaN,NaN,NaN,247224.041550,13353.916707,13513.247779,13513.247779
3,A015760,한국전력,유틸리티,전력,코스피,44400,285032,-0.448430,2.896871,-3.687636,...,29133.61,13654.365561,17147.707040,NaN,NaN,NaN,147316.591167,24639.338148,13654.365561,13654.365561
4,A035420,NAVER,IT서비스,인터넷서비스,코스피,794000,261724,0.506329,1.925546,-1.488834,...,1945.48,2117.854524,2221.450690,NaN,NaN,NaN,10916.344137,3030.094202,2117.854524,2117.854524


In [4]:
test_data = pd.read_excel('C:/KYU/2019.06.xlsx', encoding= "utf-8")

In [5]:
test_data.head()

,Code,회사명,업종\n(대),업종\n(소),코스피코스닥,주가\n(원),시가총액\n(억),상장주식수\n(만주),자사주\n(만주),자사주\n비중\n(%),...,19년 1Q.2,19년 2Q(E).2,19년 3Q(E).2,19년2Q \n매출액,19년2Q \n영업이익,19년2Q \n순이익,19년2Q \n매출액.1,19년2Q \n영업이익.1,19년2Q \n순이익.1,19년2Q \n순이익.2
0,A005930,삼성전자,반도체 관련장비 및 부품,종합 반도체,코스피,43900,2620735,596978.0,0,0.0,...,51075,48994.0,55310.0,NaN,NaN,NaN,542996.0,60531.0,48994.0,48994.0
1,A000660,SK하이닉스,반도체 관련장비 및 부품,종합 반도체,코스피,63700,463738,72800.0,4400,6.0,...,11028,6135.0,5005.0,NaN,NaN,NaN,64754.0,7557.0,6135.0,6135.0
2,A051910,LG화학,화학,종합 화학,코스피,348500,246014,7059.0,165,2.3,...,1945,2712.0,3406.0,NaN,NaN,NaN,76135.0,3991.0,2712.0,2712.0
3,A005380,현대차,자동차 및 관련부품,완성차,코스피,140000,299135,21367.0,939,4.4,...,8295,9854.0,9019.0,NaN,NaN,NaN,262152.0,11089.0,9854.0,9854.0
4,A068270,셀트리온,제약 및 바이오,바이오,코스피,205000,263075,12833.0,85,0.7,...,639,839.0,1155.0,NaN,NaN,NaN,2680.0,995.0,839.0,839.0


In [6]:
basic_bis = set(standard_data['Code'])

In [7]:
test_bis = set(test_data['Code'])

In [8]:
drop_bis = test_bis - basic_bis  #신규 발행 회사 제거

In [9]:
standard_data = standard_data.set_index("Code")
standard_data.head()
test_data = test_data.set_index("Code")
test_data.head()

,회사명,업종\n(대),업종\n(소),코스피코스닥,주가\n(원),시가총액\n(억),상장주식수\n(만주),자사주\n(만주),자사주\n비중\n(%),주식수\n증가율\n(%),...,19년 1Q.2,19년 2Q(E).2,19년 3Q(E).2,19년2Q \n매출액,19년2Q \n영업이익,19년2Q \n순이익,19년2Q \n매출액.1,19년2Q \n영업이익.1,19년2Q \n순이익.1,19년2Q \n순이익.2
Code,,,,,,,,,,,,,,,,,,,,,
A005930,삼성전자,반도체 관련장비 및 부품,종합 반도체,코스피,43900,2620735,596978.0,0,0.0,-7.0,...,51075,48994.0,55310.0,NaN,NaN,NaN,542996.0,60531.0,48994.0,48994.0
A000660,SK하이닉스,반도체 관련장비 및 부품,종합 반도체,코스피,63700,463738,72800.0,4400,6.0,0.0,...,11028,6135.0,5005.0,NaN,NaN,NaN,64754.0,7557.0,6135.0,6135.0
A051910,LG화학,화학,종합 화학,코스피,348500,246014,7059.0,165,2.3,0.0,...,1945,2712.0,3406.0,NaN,NaN,NaN,76135.0,3991.0,2712.0,2712.0
A005380,현대차,자동차 및 관련부품,완성차,코스피,140000,299135,21367.0,939,4.4,-3.0,...,8295,9854.0,9019.0,NaN,NaN,NaN,262152.0,11089.0,9854.0,9854.0
A068270,셀트리온,제약 및 바이오,바이오,코스피,205000,263075,12833.0,85,0.7,0.5,...,639,839.0,1155.0,NaN,NaN,NaN,2680.0,995.0,839.0,839.0


In [10]:
for k in drop_bis:
    changed_data = test_data.drop([k])

changed_data.head()

,회사명,업종\n(대),업종\n(소),코스피코스닥,주가\n(원),시가총액\n(억),상장주식수\n(만주),자사주\n(만주),자사주\n비중\n(%),주식수\n증가율\n(%),...,19년 1Q.2,19년 2Q(E).2,19년 3Q(E).2,19년2Q \n매출액,19년2Q \n영업이익,19년2Q \n순이익,19년2Q \n매출액.1,19년2Q \n영업이익.1,19년2Q \n순이익.1,19년2Q \n순이익.2
Code,,,,,,,,,,,,,,,,,,,,,
A005930,삼성전자,반도체 관련장비 및 부품,종합 반도체,코스피,43900,2620735,596978.0,0,0.0,-7.0,...,51075,48994.0,55310.0,NaN,NaN,NaN,542996.0,60531.0,48994.0,48994.0
A000660,SK하이닉스,반도체 관련장비 및 부품,종합 반도체,코스피,63700,463738,72800.0,4400,6.0,0.0,...,11028,6135.0,5005.0,NaN,NaN,NaN,64754.0,7557.0,6135.0,6135.0
A051910,LG화학,화학,종합 화학,코스피,348500,246014,7059.0,165,2.3,0.0,...,1945,2712.0,3406.0,NaN,NaN,NaN,76135.0,3991.0,2712.0,2712.0
A005380,현대차,자동차 및 관련부품,완성차,코스피,140000,299135,21367.0,939,4.4,-3.0,...,8295,9854.0,9019.0,NaN,NaN,NaN,262152.0,11089.0,9854.0,9854.0
A068270,셀트리온,제약 및 바이오,바이오,코스피,205000,263075,12833.0,85,0.7,0.5,...,639,839.0,1155.0,NaN,NaN,NaN,2680.0,995.0,839.0,839.0


In [18]:
#제대로 기업이 빠졌는지 확인

s = set(standard_data.index.tolist())
c = set(changed_data.index.tolist())
if s - c != 0:
    print('Fail!')
else:
    print('Success')

Fail!
